In [1]:
from pathlib import Path

root = Path()
children = list(root.iterdir())
course_dirs = list(
    filter(
        lambda x: x.is_dir() and x.name not in [".git", ".DS_Store"],
        children,
    )
)
course_names = list(map(lambda x: x.name, course_dirs))

In [2]:
%pip install 'unstructured'
%pip install 'unstructured[pdf]'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip list | grep unstructured

unstructured              0.11.2
unstructured-client       0.15.0
unstructured-inference    0.7.15
unstructured.pytesseract  0.3.12
Note: you may need to restart the kernel to use updated packages.


In [4]:
# from unstructured.partition.auto import partition


In [5]:
import unstructured.partition.pdf as pdf
# from unstructured.partition.pdf import partition_pdf


/home/reaganjlee/.conda/envs/ccs-env2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/reaganjlee/.conda/envs/ccs-env2/lib/python3.11/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 101: invalid device ordinal (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
import inspect 
print(inspect.signature(pdf.partition_pdf))

(filename: str = '', file: Union[BinaryIO, tempfile.SpooledTemporaryFile, NoneType] = None, include_page_breaks: bool = False, strategy: str = 'auto', infer_table_structure: bool = False, ocr_languages: Optional[str] = None, languages: Optional[List[str]] = None, include_metadata: bool = True, metadata_filename: Optional[str] = None, metadata_last_modified: Optional[str] = None, chunking_strategy: Optional[str] = None, links: Sequence[unstructured.documents.elements.Link] = [], extract_images_in_pdf: bool = False, image_output_dir_path: Optional[str] = None, **kwargs) -> List[unstructured.documents.elements.Element]


In [96]:
base_name = "test"
def get_file_name(ending="", base_name=base_name):
    return f"{base_name}{ending}"


In [95]:
partitioned_elements = pdf.partition_pdf(
    get_file_name(".pdf"),
    include_page_breaks=True,
    strategy="hi_res",
    infer_table_structure=True,
    extract_images_in_pdf=True,
    output_type="dict",
    image_output_dir_path="testpdf_images",
)

In [91]:
len(partitioned_elements)

230

In [16]:
partitioned_elements[:5]

In [19]:
partitioned_elements[1].metadata.coordinates.points

((1260.1981201171875, 169.6866455078125),
 (1260.1981201171875, 252.9463277777776),
 (1523.743408203125, 252.9463277777776),
 (1523.743408203125, 169.6866455078125))

In [14]:
print(type(partitioned_elements))
print(partitioned_elements[10])
print(type(partitioned_elements[10]))

<class 'list'>
First name Last name SID First and last name of student to your left First and last name of student to your right
<class 'unstructured.documents.elements.Table'>


In [10]:
for i in range(10, 25):
    print(partitioned_elements[i])
    print()

First name Last name SID First and last name of student to your left First and last name of student to your right

1



Q1. [60 pts] Multiple Answer

Fill in the bubbles for ALL correct choices: there may be more than one correct choice, but there is always at least one correct choice. NO partial credit: the set of all correct answers must be checked.

1+exp θ ) for some θ ∈ R. What is the MLE estimator of θ? 1 (a) [3 pts] Let X ∼ Bernoulli(

(cid:13) X (cid:13) 0 (cid:13) 1

(cid:13) Does not exist.

(b) [3 pts] Let Y ∼ N(Xθ, In) for some unknown θ ∈ Rd and some known X ∈ Rn×d that has full column rank and d < n. What is the MLE estimator of θ?

(cid:13) Y + Z ∀Z ∈ Null(X)

(cid:13) (X(cid:62)X)−1X(cid:62)Y (cid:13) X(cid:62)(XX(cid:62))−1Y (cid:13) Does not exist.

(c) [3 pts] Let f (x) = − (cid:80)n

i=1 xi log xi. For some x such that (cid:80)n

i=1 xi = 1 and xi > 0, the Hessian of f is:

(c) [3 pts] Let f(x) = — 0, x log x;. For some x such that >\7_, x; = 1 and x; > 0, the Hessi

### Get Image from coordinates 


In [51]:
element = partitioned_elements[1]
print(element.metadata.coordinates.to_dict())
print(element.metadata.coordinates.system)
print(element.metadata.coordinates.system.orientation)
print(element.metadata.coordinates.system.width)
print(element.metadata.coordinates.system.height)


{'points': ((1260.1981201171875, 169.6866455078125), (1260.1981201171875, 252.9463277777776), (1523.743408203125, 252.9463277777776), (1523.743408203125, 169.6866455078125)), 'system': 'PixelSpace', 'layout_width': 1700, 'layout_height': 2200}
Orientation.SCREEN
1700
2200


In [112]:
import fitz  # PyMuPDF

def convert_pdf_to_jpg_with_quad(element, pdf_path, output_path, debug=False):
    """
    Convert a PDF to JPG after cropping with given quadrilateral points, scaling them to fit the PDF page dimensions.
    :param pdf_path: Path to the PDF file.
    :param quad_points: A list of 4 tuples, each representing the corners of the quadrilateral.
    :param output_path: Path to save the output JPG file.
    :param pixel_sys_width: Width of the original pixel-based system.
    :param pixel_sys_height: Height of the original pixel-based system.
    """
    quad_points = element.metadata.coordinates.points
    pixel_sys_width = element.metadata.coordinates.system.width
    pixel_sys_height = element.metadata.coordinates.system.height
    pdf_page_num = element.metadata.page_number


    doc = fitz.open(pdf_path)
    page = doc.load_page(pdf_page_num-1)
    
    # Get the dimensions of the PDF page in points
    pdf_width, pdf_height = page.rect.width, page.rect.height
    if debug:
        print("beginning page cropbox: ", page.cropbox)
        print(f"pdf_width: {pdf_width}")
        print(f"pdf_height: {pdf_height}")

    # Calculate scaling factors
    scale_x = pdf_width / pixel_sys_width 
    scale_y = pdf_height / pixel_sys_height

    # Transform and scale the coordinates
    transformed_quad_points = []
    for x, y in quad_points:
        x_scaled = x * scale_x
        y_scaled = y * scale_y

        # Consider leaving this to raise if out of bounds 
        x_scaled = min(max(x_scaled, 0), pdf_width)
        y_scaled = min(max(y_scaled, 0), pdf_height)

        transformed_quad_points.append((x_scaled, y_scaled))


    # Convert quadrilateral into a rectangle for cropping (minimal bounding box)
    rect = fitz.Rect(fitz.Point(*transformed_quad_points[0]), fitz.Point(*transformed_quad_points[2]))

    if debug:
        print("original quad_points", quad_points)
        print("transformed_quad_points", transformed_quad_points)
        print("pdf dimensions", pdf_width, pdf_height)
        print("after page cropbox: ", page.cropbox)
        print("after page mediabox: ", page.mediabox)
        print("rect is: ", rect)

    # Crop the page
    page.set_cropbox(rect)

    zoom_x = 2.0  # horizontal zoom
    zoom_y = 2.0  # vertical zoom
    mat = fitz.Matrix(zoom_x, zoom_y)  # zoom factor 2 in each dimension
    pix = page.get_pixmap(matrix=mat)  # use 'mat' instead of the identity matrix

    # Save the image as JPG
    pix.save(output_path)

    # Close the document
    doc.close()

# Example usage
element = partitioned_elements[150]
pdf_path = get_file_name(".pdf")
output_path = f'{get_file_name()}_output.jpg'

convert_pdf_to_jpg_with_quad(element, pdf_path, output_path)

In [104]:
partitioned_elements[200].metadata.page_number


14